In [4]:
import pandas as pd
from pandas import DataFrame as df
import numpy as np
import ta

In [5]:
file_path = "C:/Users/limxu/OneDrive/Documents/UCB/SAAS/Project/data/stock_data_basic.csv"
stock_df = pd.read_csv(file_path)
print(stock_df.head())

        date   open   high    low  close    volume
0  6/15/2011  20.00  26.00  17.35  17.42  42152400
1  6/16/2011  16.99  16.99  12.99  13.26  15300400
2  6/17/2011  13.00  13.60  12.16  13.40   4813900
3  6/20/2011  13.36  15.39  13.27  14.61   3073000
4  6/21/2011  15.22  15.35  13.27  13.50   3316100


In [6]:
#check for missing data
stock_df.isna().sum()

#ensuring correct data_type
stock_df['date']=pd.to_datetime(stock_df['date'], format="%m/%d/%Y")
stock_df.dtypes

date      datetime64[ns]
open             float64
high             float64
low              float64
close            float64
volume             int64
dtype: object

### Adding y column - price change 


In [7]:
def getPrevClose(stock_df, n, i):
    row = i-1
    if row <0:
        return np.nan
    return stock_df.loc[row]["close"]

def getROC(close_px, n, stock_df, i):
    prev_close_px = getPrevClose(stock_df, n, i)
    roc = ((close_px - prev_close_px)/prev_close_px)*100
    return roc

l1=[]
n = 1


for i in range(stock_df.tail(1).index.item()+1):
    
    close_px=stock_df.loc[i]["close"]
    roc = getROC(close_px, n, stock_df, i)
    
    l1.append(roc)
stock_df["change"]=l1
print(stock_df.head())

        date   open   high    low  close    volume     change
0 2011-06-15  20.00  26.00  17.35  17.42  42152400        NaN
1 2011-06-16  16.99  16.99  12.99  13.26  15300400 -23.880597
2 2011-06-17  13.00  13.60  12.16  13.40   4813900   1.055807
3 2011-06-20  13.36  15.39  13.27  14.61   3073000   9.029851
4 2011-06-21  15.22  15.35  13.27  13.50   3316100  -7.597536


### Add 3 previous day changes

In [8]:
stock_df['minus1_change']=stock_df['change'].shift(1)
stock_df['minus2_change']=stock_df['change'].shift(2)
stock_df['minus3_change']=stock_df['change'].shift(3)
stock_df['plus1_change']=stock_df['change'].shift(-1)

In [9]:
stock_df['plus1_open']=stock_df['open'].shift(-1)
stock_df['plus1_close']=stock_df['close'].shift(-1)

In [10]:
stock_df.head()

,date,open,high,low,close,volume,change,minus1_change,minus2_change,minus3_change,plus1_change,plus1_open,plus1_close
0,2011-06-15,20.00,26.00,17.35,17.42,42152400,NaN,NaN,NaN,NaN,-23.880597,16.99,13.26
1,2011-06-16,16.99,16.99,12.99,13.26,15300400,-23.880597,NaN,NaN,NaN,1.055807,13.00,13.40
2,2011-06-17,13.00,13.60,12.16,13.40,4813900,1.055807,-23.880597,NaN,NaN,9.029851,13.36,14.61
3,2011-06-20,13.36,15.39,13.27,14.61,3073000,9.029851,1.055807,-23.880597,NaN,-7.597536,15.22,13.50
4,2011-06-21,15.22,15.35,13.27,13.50,3316100,-7.597536,9.029851,1.055807,-23.880597,-1.185185,14.07,13.34


### (new) Add necessary tech indicators

In [11]:
def removeZero(df, col_name):
    ini_len = len(df)
    
    #drop na
    df = df.dropna()
    
    #drop zero
    index_with_zero = df[df[col_name]==0].index
    num_zero = len(index_with_zero)
    df = df.drop(index=list(index_with_zero))
    
    total_drop = ini_len - len(df)
    
    print("for col " + col_name,  "num of zero is ", num_zero, "total dropped is ", total_drop)
    
    df = df.reset_index(drop=True)
    
    return df

In [12]:
high = stock_df.high
low = stock_df.low
close = stock_df.close
volume = stock_df.volume

In [13]:
#volatility indicator
n=14
stock_df['atr']=ta.volatility.average_true_range(high, low, close, n, fillna=False)

In [14]:
#trend indicator
stock_df['macd']=ta.trend.macd(close, n_fast=12, n_slow=26, fillna=False)

In [15]:
#volume indicator
stock_df['accdist']=ta.volume.acc_dist_index(high, low, close, volume, fillna=False)
print(sum(stock_df.accdist==0) + stock_df.accdist.isna().sum())

0


In [16]:
#momentum indicator

#these functions chooses a period of n days
#eg. if n=10, get dif of day1 and day10, not day0 and day10

#these functions chooses a period of n days
#eg. if n=10, get dif of day1 and day10, not day0 and day10

def getPrevClose(stock_df, n, i):
    row = i+1 -n
    return stock_df.loc[row]["close"]

def getROC(close_px, n, stock_df, i):
    prev_close_px = getPrevClose(stock_df, n, i)
    roc = ((close_px - prev_close_px)/prev_close_px)*100
    return roc

#What to do for the first n-1 days hmm
#I think for now since we looking at short term, just take the earliest possible dates

#Add ROC
n = 12
stock_df["roc"]=np.nan

for i in range(stock_df.tail(1).index.item()+1):
    
    close_px=stock_df.loc[i]["close"]
    
    if i<n-1:#for the first n-1 entries, get ROC wrt 1st entry
        roc = getROC(close_px, i+1, stock_df, i)
    
    else:#for all other entries
        roc = getROC(close_px, n, stock_df, i)
        
    stock_df.loc[i, "roc"] = roc
print(stock_df.head())

        date   open   high    low  close    volume     change  minus1_change  \
0 2011-06-15  20.00  26.00  17.35  17.42  42152400        NaN            NaN   
1 2011-06-16  16.99  16.99  12.99  13.26  15300400 -23.880597            NaN   
2 2011-06-17  13.00  13.60  12.16  13.40   4813900   1.055807     -23.880597   
3 2011-06-20  13.36  15.39  13.27  14.61   3073000   9.029851       1.055807   
4 2011-06-21  15.22  15.35  13.27  13.50   3316100  -7.597536       9.029851   

   minus2_change  minus3_change  plus1_change  plus1_open  plus1_close  atr  \
0            NaN            NaN    -23.880597       16.99        13.26  0.0   
1            NaN            NaN      1.055807       13.00        13.40  0.0   
2            NaN            NaN      9.029851       13.36        14.61  0.0   
3     -23.880597            NaN     -7.597536       15.22        13.50  0.0   
4       1.055807     -23.880597     -1.185185       14.07        13.34  0.0   

   macd       accdist        roc  
0   NaN -

### Cleaning

In [18]:
stock_df.head()

,date,open,high,low,close,volume,change,minus1_change,minus2_change,minus3_change,plus1_change,plus1_open,plus1_close,atr,macd,accdist,roc
0,2011-06-15,20.00,26.00,17.35,17.42,42152400,NaN,NaN,NaN,NaN,-23.880597,16.99,13.26,0.0,NaN,-4.147016e+07,0.000000
1,2011-06-16,16.99,16.99,12.99,13.26,15300400,-23.880597,NaN,NaN,NaN,1.055807,13.00,13.40,0.0,NaN,-5.470501e+07,-23.880597
2,2011-06-17,13.00,13.60,12.16,13.40,4813900,1.055807,-23.880597,NaN,NaN,9.029851,13.36,14.61,0.0,NaN,-5.122831e+07,-23.076923
3,2011-06-20,13.36,15.39,13.27,14.61,3073000,9.029851,1.055807,-23.880597,NaN,-7.597536,15.22,13.50,0.0,NaN,-5.041657e+07,-16.130884
4,2011-06-21,15.22,15.35,13.27,13.50,3316100,-7.597536,9.029851,1.055807,-23.880597,-1.185185,14.07,13.34,0.0,NaN,-5.299930e+07,-22.502870


In [19]:
new_features = ['atr','macd','accdist','roc']
for col in new_features:
    stock_df = removeZero(stock_df, col)
    
stock_df = stock_df.dropna()

print(stock_df.head())

for col atr num of zero is  0 total dropped is  26
for col macd num of zero is  0 total dropped is  0
for col accdist num of zero is  0 total dropped is  0
for col roc num of zero is  6 total dropped is  6
        date   open   high    low  close   volume    change  minus1_change  \
0 2011-07-21  18.15  19.06  17.96  18.76   892900  5.393258       3.308183   
1 2011-07-22  18.60  18.60  17.85  18.03   554500 -3.891258       5.393258   
2 2011-07-25  18.75  18.75  17.50  17.81  1069500 -1.220189      -3.891258   
3 2011-07-26  18.00  18.13  17.20  17.28   834200 -2.975856      -1.220189   
4 2011-07-27  17.20  17.25  16.50  16.62   532800 -3.819444      -2.975856   

   minus2_change  minus3_change  plus1_change  plus1_open  plus1_close  \
0       1.651917      -4.668166     -3.891258       18.60        18.03   
1       3.308183       1.651917     -1.220189       18.75        17.81   
2       5.393258       3.308183     -2.975856       18.00        17.28   
3      -3.891258       5.3932

### Saving

In [21]:
print(stock_df.head())
print(stock_df.isna().sum())

        date   open   high    low  close   volume    change  minus1_change  \
0 2011-07-21  18.15  19.06  17.96  18.76   892900  5.393258       3.308183   
1 2011-07-22  18.60  18.60  17.85  18.03   554500 -3.891258       5.393258   
2 2011-07-25  18.75  18.75  17.50  17.81  1069500 -1.220189      -3.891258   
3 2011-07-26  18.00  18.13  17.20  17.28   834200 -2.975856      -1.220189   
4 2011-07-27  17.20  17.25  16.50  16.62   532800 -3.819444      -2.975856   

   minus2_change  minus3_change  plus1_change  plus1_open  plus1_close  \
0       1.651917      -4.668166     -3.891258       18.60        18.03   
1       3.308183       1.651917     -1.220189       18.75        17.81   
2       5.393258       3.308183     -2.975856       18.00        17.28   
3      -3.891258       5.393258     -3.819444       17.20        16.62   
4      -1.220189      -3.891258     -2.647413       16.58        16.18   

        atr      macd       accdist        roc  
0  1.622551  0.304654 -5.036333e+07  

In [22]:
file_name = "stock_df_full.csv"
path = "C:/Users/limxu/OneDrive/Documents/UCB/SAAS/Project/data/" + file_name
stock_df.to_csv(path)